In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')
from transformers import TFAutoModel
from tqdm import tqdm
from keras.utils import np_utils
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.0 MB/s eta 0:00:00
Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 103.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing instal

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be DOWNGRADED:
  libcudnn8
0 upgraded, 0 newly installed, 1 downgraded, 1 to remove and 22 not upgraded.
Need to get 430 MB of archives.
After this operation, 1,153 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 5s (87.9 MB/s)
(Reading database ... 122352 files and directories currently installed.)
Removing libcudnn8-dev (8.7.0.84-1+cuda11.8) ...
update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
dpkg: warning: downgrading libcudnn8 from 8.7.0.84-1+cuda11.8 to 8.1.0.77-1+cuda11.2
(Reading database ... 122319 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.7

In [ ]:
from keras import backend as K
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout,Reshape,Softmax,LSTM, Dropout, Bidirectional
from keras.models import Model
import numpy as np
BERT = TFAutoModel.from_pretrained('bert-base-uncased')
for layer in BERT.layers:
    layer.trainable = False #使BERT层的参数不参与反相传播
def embedding(inputs_id):
    if tf.test.is_gpu_available():
        device = "/gpu:0"
    else:
        device = "/cpu:0"
    with tf.device(device):
      outputs = BERT(inputs_id, output_hidden_states=True, training=False)
      hidden = outputs[2]
      embeddings = tf.stack(hidden, axis=0)
      final = tf.reduce_sum(embeddings[-4:], axis=0)
    del hidden,embeddings,outputs
    return final
def Siamese():
    input_shape = (50,)
    input_dtype = tf.int32
    input = tf.keras.layers.Input(shape=input_shape, dtype=input_dtype)
    X_1 = embedding(input)
    X_1_1 = Conv1D(200, kernel_size=1, strides=1, activation='relu')(X_1)
    X_1_1 = tf.keras.layers.BatchNormalization()(X_1_1)
    X_1_1 = MaxPooling1D(pool_size=300,padding='same')(X_1_1)
    X_1_1 = Flatten()(X_1_1)

    X_1_2 = Conv1D(200, kernel_size=2, strides=1, activation='relu')(X_1)
    X_1_2 = tf.keras.layers.BatchNormalization()(X_1_2)
    X_1_2 = MaxPooling1D(pool_size=299,padding='same')(X_1_2)
    X_1_2 = Flatten()(X_1_2)

    X_1_3 = Conv1D(200, kernel_size=3, strides=1, activation='relu')(X_1)
    X_1_3 = tf.keras.layers.BatchNormalization()(X_1_3)
    X_1_3 = MaxPooling1D(pool_size=298,padding='same')(X_1_3)
    X_1_3 = Flatten()(X_1_3)

    X_1 = tf.keras.layers.Concatenate(axis=-1)([X_1_1, X_1_2])
    output = tf.keras.layers.Concatenate(axis=-1)([X_1, X_1_3])
    # output = lstm_layer(X_1)
    output = Dropout(0.6)(output)
    output = Dense(300, activation='relu')(output)
    output = tf.keras.layers.BatchNormalization(axis=-1)(output)

    output = Dropout(0.4)(output)
    output = Dense(100, activation='relu')(output)
    output = tf.keras.layers.BatchNormalization(axis=-1)(output)


    sub_model = Model(input, output)

    # Then define the tell-digits-apart model
    left = Input(shape=input_shape)
    right = Input(shape=input_shape)

    # The vision model will be shared, weights and all
    left_output = sub_model(left)
    right_output = sub_model(right)

    out =  tf.keras.layers.Concatenate(axis=-1)([left_output,right_output])
    prediction = tf.keras.layers.Dense(2, activation='softmax')(out)

    siamese_model = Model([left, right], prediction)
    return siamese_model

DBRD = Siamese()
DBRD.summary()






Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 model (Functional)             (None, 100)          110618840   ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 concatenate_2 (Concatenate)    (None, 200)          0           ['model[0][0]',            

In [ ]:
import pickle
left = tf.convert_to_tensor(torch.load('/content/drive/MyDrive/data/token50/eclipse_token1.pt'))
right = tf.convert_to_tensor(torch.load('/content/drive/MyDrive/data/token50/eclipse_token2.pt'))

with open('/content/drive/MyDrive/data/label/ec_l.pkl', 'rb') as f:
    label = pickle.load(f)
label = [0 if i == -1 else i for i in label]
label = tf.convert_to_tensor(label, dtype=tf.int32)
label = np_utils.to_categorical(label, 2)
with open('/content/drive/MyDrive/data/eclipse_normal/eclipse_ct.pkl', 'rb') as f:
    p1 = pickle.load(f)
    p1 = tf.convert_to_tensor(p1)
with open('/content/drive/MyDrive/data/eclipse_normal/eclipse_pr.pkl', 'rb') as f:
    p2 = pickle.load(f)
    p2 = tf.convert_to_tensor(p2)
with open('/content/drive/MyDrive/data/eclipse_normal/eclipse_sv.pkl', 'rb') as f:
    p3 = pickle.load(f)
    p3 = tf.convert_to_tensor(p3)

RANDOM_SEED =22
left = tf.random.shuffle(left, seed=RANDOM_SEED )
right = tf.random.shuffle(right, seed=RANDOM_SEED )
labels = tf.random.shuffle(label, seed=RANDOM_SEED )
p1 = tf.random.shuffle(p1, seed=RANDOM_SEED )
p2 = tf.random.shuffle(p2, seed=RANDOM_SEED )
p3 = tf.random.shuffle(p3, seed=RANDOM_SEED )


In [ ]:
data_size = len(labels)
train_size = int(0.8 * data_size)
val_size = int(0.1 * data_size)
test_size = data_size - train_size - val_size

split_left = tf.split(left, [train_size, val_size, test_size], axis=0)
split_right = tf.split(right, [train_size, val_size, test_size], axis=0)
split_p1 = tf.split(p1, [train_size, val_size, test_size], axis=0)
split_p2 = tf.split(p2, [train_size, val_size, test_size], axis=0)
split_p3 = tf.split(p3, [train_size, val_size, test_size], axis=0)
split_labels = tf.split(labels, [train_size, val_size, test_size], axis=0)

train_left = split_left[0]
train_right = split_right[0]
train_label = split_labels[0]
train_p1 = split_p1[0]
train_p2 = split_p2[0]
train_p3 = split_p3[0]

val_left = split_left[1]
val_right = split_right[1]
val_label = split_labels[1]
val_p1 = split_p1[1]
val_p2 = split_p2[1]
val_p3 = split_p3[1]


test_left = split_left[2]
test_right = split_right[2]
test_label = split_labels[2]
test_p1 = split_p1[2]
test_p2 = split_p2[2]
test_p3 = split_p3[2]

In [ ]:
# train_left = train_left[:500]
# train_right = train_right[:500]
# train_label = train_label[:500]
# train_p1 = train_p1[:500]
# train_p2 = train_p2[:500]
# train_p3 = train_p3[:500]

# val_left = val_left[:500]
# val_right = val_right[:500]
# val_label = val_label[:500]
# val_p1 = val_p1[:500]
# val_p2 = val_p2[:500]
# val_p3 = val_p3[:500]

In [ ]:

TP = tf.keras.metrics.TruePositives()
TN = tf.keras.metrics.TrueNegatives()
FP = tf.keras.metrics.FalsePositives()
FN = tf.keras.metrics.FalseNegatives()
def evaluate(left, right, labels, model, loss_fn, p1, p2, p3, batch_size):

    for i in tqdm(range(0, len(left), batch_size)):
        val_left_batch = left[i:i+batch_size]
        val_right_batch = right[i:i+batch_size]
        val_labels_batch = labels[i:i+batch_size]
        val_p1_batch = p1[i:i+batch_size]
        val_p2_batch = p2[i:i+batch_size]
        val_p3_batch = p3[i:i+batch_size]
        val_pred = model([val_left_batch, val_right_batch])
        val_loss = loss_fn(val_labels_batch,val_pred)+ tf.dtypes.cast(tf.math.reduce_mean(val_p1_batch),tf.float32)+ tf.dtypes.cast(tf.math.reduce_mean(val_p2_batch),tf.float32) + tf.dtypes.cast(tf.math.reduce_mean(val_p3_batch),tf.float32)
        val_accuracy_metric.update_state(val_labels_batch, val_pred)

        TP.update_state(val_labels_batch[:, 0], val_pred[:, 0])
        TN.update_state(val_labels_batch[:, 0], val_pred[:, 0])
        FP.update_state(val_labels_batch[:, 0], val_pred[:, 0])
        FN.update_state(val_labels_batch[:, 0], val_pred[:, 0])
        val_loss_tracker.update_state(val_loss)
    print("TP:",TP.result().numpy())
    print("TN:",TN.result().numpy())
    print("FP:",FP.result().numpy())
    print("FN:",FN.result().numpy())
    TP.reset_states()
    TN.reset_states()
    FP.reset_states()
    FN.reset_states()

#额外训练参数
# w1 = tf.Variable(tf.random.normal([1], mean=0.0, stddev=1.0))
# w2 = tf.Variable(tf.random.normal([1], mean=0.0, stddev=1.0))
# w3 = tf.Variable(tf.random.normal([1], mean=0.0, stddev=1.0))
loss_tracker = tf.keras.metrics.Mean(name="loss")
val_loss_tracker = tf.keras.metrics.Mean(name="loss")
accuracy_metric = tf.keras.metrics.BinaryAccuracy()
val_accuracy_metric = tf.keras.metrics.BinaryAccuracy()

@tf.function
def train_step(inputs, labels, model, optimizer, loss_fn, p1,p2,p3):#再加入一个验证集
  if tf.test.is_gpu_available():
    device = "/gpu:0"
  else:
    device = "/cpu:0"
  with tf.device(device):
    with tf.GradientTape() as tape:
        left, right = inputs
        pred = model([left, right])
        loss = loss_fn(labels, pred) + tf.dtypes.cast(tf.math.reduce_mean(p1),tf.float32)+ tf.dtypes.cast(tf.math.reduce_mean(p2),tf.float32) + tf.dtypes.cast(tf.math.reduce_mean(p3),tf.float32)
    # Compute gradients and update weights
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    # Update metrics
    accuracy_metric.update_state(labels, pred)
    loss_tracker.update_state(loss)



# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)




# Train the model
epochs = 100
batch_size = 64
patience = 5
wait = 0
best = 0
best_loss = 100

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)



# Train the model
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    for i in tqdm(range(0, len(train_left), batch_size)):
        left_batch = train_left[i:i+batch_size]
        right_batch = train_right[i:i+batch_size]
        labels_batch = train_label[i:i+batch_size]
        p1_batch = train_p1[i:i+batch_size]
        p2_batch = train_p2[i:i+batch_size]
        p3_batch = train_p3[i:i+batch_size]

        # Train on batch
        train_step([left_batch, right_batch], labels_batch, DBRD, optimizer, loss_fn, p1_batch, p2_batch, p3_batch)

    # Evaluate on validation set
    evaluate(val_left, val_right, val_label, DBRD, loss_fn, val_p1, val_p2, val_p3, batch_size)

    # Print training progress
    print("Training accuracy:", accuracy_metric.result().numpy())
    print("Training loss:", loss_tracker.result().numpy())
    print("Validation accuracy:", val_accuracy_metric.result().numpy())
    print("Validation loss:", val_loss_tracker.result().numpy())
    val_loss = val_loss_tracker.result()
    val_acc = val_accuracy_metric.result()
    # Reset metrics at the end of each epoch
    accuracy_metric.reset_states()
    loss_tracker.reset_states()
    val_loss_tracker.reset_states()
    val_accuracy_metric.reset_states()

    wait += 1

    if val_acc > best:
      best = val_acc
      wait = 0
    if val_loss < best_loss:
      best_loss = val_loss
      wait = 0

    if wait >= patience:
      break
    print("Best acc:",best)
    print("Best loss:",best_loss)
    print("Wait:",wait)
# Save the model
DBRD.save("model.h5")

Epoch 1/100


  0%|          | 0/3091 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  
Cause: Unable to locate the source code of <bound method _BaseOptimizer._update_step_xla of <tensorflow.python.eager.polymorphic_function.tracing_compiler.TfMethodTarget object at 0x7efcb1a8fb50>>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <bound method _BaseOptimizer._update_step_xla of <tensorflow.python.eager.polymorphic_function.tracing_compiler.TfMethodTarget object at 0x7efcb1a8fb50>>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


100%|██████████| 387/387 [02:44<00:00,  2.35it/s]


TP: 14990.0
TN: 3304.0
FP: 5521.0
FN: 911.0
Training accuracy: 0.72559667
Training loss: 1.0106362
Validation accuracy: 0.73986894
Validation loss: 0.98248357
Best acc: tf.Tensor(0.73986894, shape=(), dtype=float32)
Best loss: tf.Tensor(0.98248357, shape=(), dtype=float32)
Wait: 0
Epoch 2/100


...

TP: 12627.0
TN: 6100.0
FP: 2725.0
FN: 3274.0
Training accuracy: 0.88450086
Training loss: 0.7231615
Validation accuracy: 0.7573809
Validation loss: 1.1388571
Best acc: tf.Tensor(0.78342634, shape=(), dtype=float32)
Best loss: tf.Tensor(0.94807917, shape=(), dtype=float32)
Wait: 3
Epoch 9/100


100%|██████████| 387/387 [02:43<00:00,  2.37it/s]


TP: 11899.0
TN: 6390.0
FP: 2435.0
FN: 4002.0
Training accuracy: 0.8973161
Training loss: 0.6958683
Validation accuracy: 0.73966676
Validation loss: 1.2607397
Best acc: tf.Tensor(0.78342634, shape=(), dtype=float32)
Best loss: tf.Tensor(0.94807917, shape=(), dtype=float32)
Wait: 4
Epoch 10/100


100%|██████████| 387/387 [02:42<00:00,  2.38it/s]


TP: 13306.0
TN: 5657.0
FP: 3168.0
FN: 2595.0
Training accuracy: 0.9063601
Training loss: 0.6728688
Validation accuracy: 0.7669255
Validation loss: 1.1876303


In [ ]:
!nvidia-smi

Fri Apr 21 03:13:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

DBRD.save("/content/drive/MyDrive/model_50_ec.h5")